<a href="https://colab.research.google.com/github/magewade/ml-ds-useful-things/blob/main/Pandas_lifehacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import random

import os

import matplotlib.pyplot as plt
%matplotlib inline

Данные для данного ноутбука можно скачать здесь: https://disk.yandex.ru/d/LtCfr7Sd_Rankw

Монтируем пространство на гугл-диске.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Считываем данные

In [ ]:
%%time
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/customer_segmentation.csv', encoding = 'cp1252', nrows=50000)

print(f'Shape: {df.shape}')
df.shape
df.head(3)

Shape: (50000, 8)
CPU times: user 57.2 ms, sys: 21.1 ms, total: 78.3 ms
Wall time: 563 ms


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom


## Transform

Transform применяется для создания столбца, который содержит агрегированную по указанным полям информацию.

Например, у нас есть данные о походах клиентов в магазин. Каждый поход в магазин имеет уникальный **InvoiceNo**. И мы хотим для каждого **InvoiceNo** посчитать общее количество товаров в корзине. Более того, нам бы хотелось иметь в исходных данных столбец, в котором каждому **InvoiceNo** сопоставляется общее количество товаров, соответствующих этому **InvoiceNo**.

In [ ]:
df['QuantityByInvoiceNo'] = df.groupby(by=['InvoiceNo', 'CustomerID'])['Quantity'].transform('sum')
df.head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,QuantityByInvoiceNo
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,40.0
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,40.0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,40.0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,40.0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,40.0
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,12/1/2010 8:26,7.65,17850.0,United Kingdom,40.0
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,12/1/2010 8:26,4.25,17850.0,United Kingdom,40.0
7,536366,22633,HAND WARMER UNION JACK,6,12/1/2010 8:28,1.85,17850.0,United Kingdom,12.0
8,536366,22632,HAND WARMER RED POLKA DOT,6,12/1/2010 8:28,1.85,17850.0,United Kingdom,12.0
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,12/1/2010 8:34,1.69,13047.0,United Kingdom,83.0


## Dropna + subset

Исключение строк с пустыми значениями в определенных столбцах

In [ ]:
print(f'Размер исходного датасета: {df.shape}')

df.dropna(subset=["StockCode", "Description"], inplace=True)
print(f'Размер итогового датасета: {df.shape}')

Размер исходного датасета: (50000, 9)
Размер итогового датасета: (49858, 9)


## Различные функции агрегации

In [ ]:
def func(x):
  res = 0
  for xi in x:
    res += xi + random.randint(0, 5)
  return res

df_agg = df.groupby(by=['InvoiceNo', 'CustomerID']).agg(
    MeanQuantity=('Quantity', 'mean'),
    MedianUnitPrice=('UnitPrice', 'median'),
    P75Quantity=('Quantity', lambda x: x.quantile(0.75)),
    FuncQuantity=('Quantity', func)
).reset_index()

df_agg.head(10)

,InvoiceNo,CustomerID,MeanQuantity,MedianUnitPrice,P75Quantity,FuncQuantity
0,536365,17850.0,5.714286,3.39,6.00,62
1,536366,17850.0,6.000000,1.85,6.00,18
2,536367,13047.0,6.916667,4.60,6.00,121
3,536368,13047.0,3.750000,4.95,3.75,26
4,536369,13047.0,3.000000,5.95,3.00,7
5,536370,12583.0,22.450000,1.95,24.00,502
6,536371,13748.0,80.000000,2.55,80.00,85
7,536372,17850.0,6.000000,1.85,6.00,20
8,536373,17850.0,5.500000,3.07,6.00,121
9,536374,15100.0,32.000000,10.95,32.00,34


## Примение нескольких операций преобразования датафрейма подряд

In [ ]:
def func1(df):
  return df.drop_duplicates()

def func2(df):
  return df.dropna()

#
df2 = df.pipe(func1).pipe(func2).copy()
df2.shape

(31586, 9)

## Label Encoding

Полезно, когда мы хотим категориальную переменную закодировать числом.

In [ ]:
labels, unique_labels = pd.factorize(df['Country'])

df['CountryLabel'] = labels
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,QuantityByInvoiceNo,CountryLabel
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,40.0,0
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,40.0,0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,40.0,0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,40.0,0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,40.0,0


In [ ]:
unique_labels

Index(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal', 'Italy',
       'Belgium', 'Lithuania', 'Japan', 'Iceland', 'Channel Islands',
       'Denmark', 'Cyprus', 'Sweden', 'Austria', 'Israel', 'Finland',
       'Bahrain'],
      dtype='object')

## Explode

Обработка признаков, представленных в виде списка.

Допустим у нас есть столбец со списком емейлов. Мы хотим этот список распаковать, добавив в датафрейм соответствующее количество строк - по одной для каждого емейла.

In [ ]:
df['Email'] = [[f'email_{i+1}@gg.com' for i in range(random.randint(1, 3))] for j in range(df.shape[0])]
df.head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,QuantityByInvoiceNo,CountryLabel,Email
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,40.0,0,"[email_1@gg.com, email_2@gg.com, email_3@gg.com]"
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,40.0,0,[email_1@gg.com]
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,40.0,0,"[email_1@gg.com, email_2@gg.com, email_3@gg.com]"
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,40.0,0,"[email_1@gg.com, email_2@gg.com]"
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,40.0,0,"[email_1@gg.com, email_2@gg.com]"
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,12/1/2010 8:26,7.65,17850.0,United Kingdom,40.0,0,[email_1@gg.com]
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,12/1/2010 8:26,4.25,17850.0,United Kingdom,40.0,0,[email_1@gg.com]
7,536366,22633,HAND WARMER UNION JACK,6,12/1/2010 8:28,1.85,17850.0,United Kingdom,12.0,0,[email_1@gg.com]
8,536366,22632,HAND WARMER RED POLKA DOT,6,12/1/2010 8:28,1.85,17850.0,United Kingdom,12.0,0,"[email_1@gg.com, email_2@gg.com, email_3@gg.com]"
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,12/1/2010 8:34,1.69,13047.0,United Kingdom,83.0,0,[email_1@gg.com]


ignore_index - хотим заново проставить индексы с нуля, иначе сохраним индексы и продублируем их так же, как и остальные признаки

In [ ]:
df.explode('Email', ignore_index=True).head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,QuantityByInvoiceNo,CountryLabel,Email
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,40.0,0,email_1@gg.com
1,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,40.0,0,email_2@gg.com
2,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,40.0,0,email_3@gg.com
3,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,40.0,0,email_1@gg.com
4,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,40.0,0,email_1@gg.com
5,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,40.0,0,email_2@gg.com
6,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,40.0,0,email_3@gg.com
7,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,40.0,0,email_1@gg.com
8,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,40.0,0,email_2@gg.com
9,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,40.0,0,email_1@gg.com


## Between

Фильтруем значения признака по конкретному диапазону, указывая, какие границы включать.

inclusive - "left", "right", "both", "neither"

In [ ]:
dff = df[df.UnitPrice.between(2.55, 3.39, inclusive='left')].copy()
dff.UnitPrice.min(), dff.UnitPrice.max()

(2.55, 3.38)

In [ ]:
dff = df[df.UnitPrice.between(2.55, 3.39, inclusive='right')].copy()
dff.UnitPrice.min(), dff.UnitPrice.max()

(2.75, 3.39)

In [ ]:
dff = df[df.UnitPrice.between(2.55, 3.39, inclusive='both')].copy()
dff.UnitPrice.min(), dff.UnitPrice.max()

(2.55, 3.39)

In [ ]:
dff = df[df.UnitPrice.between(2.55, 3.39, inclusive='neither')].copy()
dff.UnitPrice.min(), dff.UnitPrice.max()

(2.75, 3.38)

In [ ]:
dff = df[(df.UnitPrice > 2.55) & (df.UnitPrice <= 3.39)].copy()
dff.UnitPrice.min(), dff.UnitPrice.max()

(2.75, 3.39)

## Discribe + T + Style

Делаем выводимую статистику более наглядной

In [ ]:
df.describe().T.drop(columns=['count']).style.highlight_min(color='#ceeeec')

,mean,std,min,25%,50%,75%,max
Quantity,8.368647,55.293005,-9360.000000,1.000000,2.000000,8.000000,2880.000000
UnitPrice,3.880882,5.825107,0.000000,1.250000,2.510000,4.250000,100.000000
CustomerID,15431.221987,1753.358819,12347.000000,14057.000000,15464.000000,17059.000000,18283.000000
QuantityByInvoiceNo,332.264267,553.010225,-9360.000000,94.000000,178.000000,350.000000,6668.000000
CountryLabel,0.356151,1.827953,0.000000,0.000000,0.000000,0.000000,23.000000
Lag1,8.368795,55.293549,-9360.000000,1.000000,2.000000,8.000000,2880.000000
Lag3,8.369010,55.294648,-9360.000000,1.000000,2.000000,8.000000,2880.000000
Lead1,8.368694,55.293558,-9360.000000,1.000000,2.000000,8.000000,2880.000000
Lead3,8.368749,55.294666,-9360.000000,1.000000,2.000000,8.000000,2880.000000


In [ ]:
df.describe().T.style.background_gradient(subset=['mean', '50%', 'min'], cmap='coolwarm')

,count,mean,std,min,25%,50%,75%,max
Quantity,49858.000000,8.368647,55.293005,-9360.000000,1.000000,2.000000,8.000000,2880.000000
UnitPrice,49858.000000,6.436641,166.741421,0.000000,1.250000,2.510000,4.250000,16888.020000
CustomerID,32119.000000,15431.221987,1753.358819,12347.000000,14057.000000,15464.000000,17059.000000,18283.000000
QuantityByInvoiceNo,32119.000000,332.264267,553.010225,-9360.000000,94.000000,178.000000,350.000000,6668.000000
CountryLabel,49858.000000,0.356151,1.827953,0.000000,0.000000,0.000000,0.000000,23.000000


## Dtypes

Работаем с типами данных

In [ ]:
df.dtypes

InvoiceNo               object
StockCode               object
Description             object
Quantity                 int64
InvoiceDate             object
UnitPrice              float64
CustomerID             float64
Country                 object
QuantityByInvoiceNo    float64
CountryLabel             int64
Email                   object
dtype: object

In [ ]:
df.convert_dtypes().dtypes

InvoiceNo               string
StockCode               string
Description             string
Quantity                 Int64
InvoiceDate             string
UnitPrice              Float64
CustomerID               Int64
Country                 string
QuantityByInvoiceNo      Int64
CountryLabel             Int64
Email                   object
dtype: object

Выбираем только числовые признаки

In [ ]:
df.select_dtypes(include=[np.number]).head(5)

,Quantity,UnitPrice,CustomerID,QuantityByInvoiceNo,CountryLabel
0,6,2.55,17850.0,40.0,0
1,6,3.39,17850.0,40.0,0
2,8,2.75,17850.0,40.0,0
3,6,3.39,17850.0,40.0,0
4,6,3.39,17850.0,40.0,0


## nlargest & nsmallest

Выбираем строки с наибольшими (наименьшими) значениями

In [ ]:
df.nlargest(3, 'Quantity')

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,QuantityByInvoiceNo,CountryLabel,Email
4945,536830,84077,WORLD WAR 2 GLIDERS ASSTD DESIGNS,2880,12/2/2010 17:38,0.18,16754.0,United Kingdom,4280.0,0,"[email_1@gg.com, email_2@gg.com]"
32671,539101,22693,GROW A FLYTRAP OR SUNFLOWER IN TIN,2400,12/16/2010 10:35,0.94,16029.0,United Kingdom,4800.0,0,"[email_1@gg.com, email_2@gg.com]"
4850,536809,84950,ASSORTED COLOUR T-LIGHT HOLDER,1824,12/2/2010 16:48,0.55,15299.0,United Kingdom,1824.0,0,"[email_1@gg.com, email_2@gg.com]"


In [ ]:
df.nsmallest(7, 'Quantity')

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,QuantityByInvoiceNo,CountryLabel,Email
4287,C536757,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,-9360,12/2/2010 14:23,0.03,15838.0,United Kingdom,-9360.0,0,[email_1@gg.com]
21518,538090,20956,?,-723,12/9/2010 14:48,0.00,NaN,United Kingdom,NaN,0,"[email_1@gg.com, email_2@gg.com, email_3@gg.com]"
34292,C539329,85123A,WHITE HANGING HEART T-LIGHT HOLDER,-500,12/17/2010 9:37,2.55,16013.0,United Kingdom,-800.0,0,[email_1@gg.com]
34291,C539329,20733,GOLD MINI TAPE MEASURE,-300,12/17/2010 9:37,0.72,16013.0,United Kingdom,-800.0,0,"[email_1@gg.com, email_2@gg.com]"
30078,C538847,85232B,SET OF 3 BABUSHKA STACKING TINS,-240,12/14/2010 13:27,4.95,17404.0,Sweden,-240.0,19,[email_1@gg.com]
4854,C536812,22574,HEART WOODEN CHRISTMAS DECORATION,-192,12/2/2010 16:58,0.72,16546.0,United Kingdom,-852.0,0,"[email_1@gg.com, email_2@gg.com, email_3@gg.com]"
4857,C536812,22588,CARD HOLDER GINGHAM HEART,-192,12/2/2010 16:58,2.10,16546.0,United Kingdom,-852.0,0,[email_1@gg.com]


## Value counts + dropna + mask

Выбираем по маске конкретные значения поля, остальные переобозначаем. Далее считаем количество строк с соответствующими значениями.

In [ ]:
df['Country'].value_counts()

United Kingdom     46880
Germany              811
France               608
EIRE                 418
Spain                174
Norway               147
Portugal             141
Belgium               99
Australia             83
Cyprus                83
Netherlands           72
Japan                 69
Switzerland           49
Italy                 42
Sweden                41
Lithuania             35
Iceland               31
Denmark               20
Channel Islands       17
Finland               17
Austria                9
Poland                 8
Israel                 2
Bahrain                2
Name: Country, dtype: int64

In [ ]:
dff = df.mask(cond=~df['Country'].isin(['United Kingdom']), other='__other__').copy()
dff['Country'].value_counts()

United Kingdom    46880
__other__          2978
Name: Country, dtype: int64

## Clip

Обрезаем маленькие и большие значения до выбранных рамок.

Например, признак принимает значения от 0 до 1000, а мы хотим, чтобы все значения больше 100 были заменены на 100.

In [ ]:
print(df['UnitPrice'].min(), df['UnitPrice'].max())

df['UnitPrice'] = df['UnitPrice'].clip(0, 100)

print(df['UnitPrice'].min(), df['UnitPrice'].max())

0.0 16888.02
0.0 100.0


## at & iat

Работает быстрее, чем loc & iloc

In [ ]:
df.at[103, 'UnitPrice'] # [index, label]

1.95

In [ ]:
df.iat[103, 5] # [index, index]

1.95

## Query

Пишем условие в виде строкового запроса

In [ ]:
a = 5
b = 10

df.query('Quantity < @a and UnitPrice > @b').head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,QuantityByInvoiceNo,CountryLabel,Email
45,536370,POST,POSTAGE,3,12/1/2010 8:45,18.00,12583.0,France,449.0,1,[email_1@gg.com]
141,C536379,D,Discount,-1,12/1/2010 9:41,27.50,14527.0,United Kingdom,-1.0,0,"[email_1@gg.com, email_2@gg.com]"
151,536382,22839,3 TIER CAKE TIN GREEN AND CREAM,2,12/1/2010 9:45,14.95,16098.0,United Kingdom,134.0,0,[email_1@gg.com]
152,536382,22838,3 TIER CAKE TIN RED AND CREAM,2,12/1/2010 9:45,14.95,16098.0,United Kingdom,134.0,0,[email_1@gg.com]
153,536382,22783,SET 3 WICKER OVAL BASKETS W LIDS,4,12/1/2010 9:45,16.95,16098.0,United Kingdom,134.0,0,[email_1@gg.com]


## Оконные функции

Движемся скользящим окном по строкам и считаем какую-нибудь агрегацию

**Rolling** - вычисления внутри окна

- window - размер окна
- min_periods - минимальное количество непустых значений, достаточное для вычисления агрегации, иначе NaN
- closed - какие границы окна включать (left, right, both, neither)


In [ ]:
df.select_dtypes(include=np.number).rolling(window=4, min_periods=3, closed='both').sum().head(10)

,Quantity,UnitPrice,CustomerID,QuantityByInvoiceNo,CountryLabel
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,20.0,8.69,53550.0,120.0,0.0
3,26.0,12.08,71400.0,160.0,0.0
4,32.0,15.47,89250.0,200.0,0.0
5,28.0,20.57,89250.0,200.0,0.0
6,28.0,21.43,89250.0,200.0,0.0
7,26.0,20.53,89250.0,172.0,0.0
8,26.0,18.99,89250.0,144.0,0.0
9,52.0,17.29,84447.0,187.0,0.0


In [ ]:
str_='fhfghf'


Можно использовать кастомную функцию для вычисления агрегации

In [ ]:
def func(x):
  return 1

df.select_dtypes(include=np.number).rolling(window=4, min_periods=3, closed='both').apply(func).head(10)

,Quantity,UnitPrice,CustomerID,QuantityByInvoiceNo,CountryLabel
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0,1.0
5,1.0,1.0,1.0,1.0,1.0
6,1.0,1.0,1.0,1.0,1.0
7,1.0,1.0,1.0,1.0,1.0
8,1.0,1.0,1.0,1.0,1.0
9,1.0,1.0,1.0,1.0,1.0


**Expanding** - накопительное вычисление (от первой строки к текущей)

In [ ]:
df.select_dtypes(include=np.number).expanding().sum().head(10)

,Quantity,UnitPrice,CustomerID,QuantityByInvoiceNo,CountryLabel,Lag1,Lag3,Lead1,Lead3
0,6.0,2.55,17850.0,40.0,0.0,NaN,NaN,6.0,6.0
1,12.0,5.94,35700.0,80.0,0.0,6.0,NaN,14.0,12.0
2,20.0,8.69,53550.0,120.0,0.0,12.0,NaN,20.0,14.0
3,26.0,12.08,71400.0,160.0,0.0,20.0,6.0,26.0,20.0
4,32.0,15.47,89250.0,200.0,0.0,26.0,12.0,28.0,26.0
5,34.0,23.12,107100.0,240.0,0.0,32.0,20.0,34.0,32.0
6,40.0,27.37,124950.0,280.0,0.0,34.0,26.0,40.0,64.0
7,46.0,29.22,142800.0,292.0,0.0,40.0,32.0,46.0,70.0
8,52.0,31.07,160650.0,304.0,0.0,46.0,34.0,78.0,76.0
9,84.0,32.76,173697.0,387.0,0.0,52.0,40.0,84.0,84.0


## Row number & Rank & Dense rank

- row number - просто порядковый номер
- rank - ранг значения с пропусками при наличии дубликатов (1, 2, 2, 4 ...)
- dense rank - тоже ранг, но без пропусков при наличии дубликатов (1, 2, 2, 3, 4 ...)

In [ ]:
df[['Quantity']].sort_values(by='Quantity', ascending=True).assign(
    row_number=df.Quantity.rank(method='first'),
    rank=df.Quantity.rank(method='min'),
    dense_rank=df.Quantity.rank(method='dense')
).head(15)

,Quantity,row_number,rank,dense_rank
4287,-9360,1.0,1.0,1.0
21518,-723,2.0,2.0,2.0
34292,-500,3.0,3.0,3.0
34291,-300,4.0,4.0,4.0
30078,-240,5.0,5.0,5.0
4857,-192,7.0,6.0,6.0
4854,-192,6.0,6.0,6.0
43663,-150,8.0,8.0,7.0
25208,-144,13.0,9.0,8.0
21699,-144,12.0,9.0,8.0


## Lead & Lag

- Lead - следующий
- Lag - предыдущий

In [ ]:
df['Lag1'] = df['Quantity'].shift(1)
df['Lag3'] = df['Quantity'].shift(3)
df['Lead1'] = df['Quantity'].shift(-1)
df['Lead3'] = df['Quantity'].shift(-3)

df[['Quantity', 'Lag1', 'Lag3', 'Lead1', 'Lead3']].head(10)

,Quantity,Lag1,Lag3,Lead1,Lead3
0,6,NaN,NaN,6.0,6.0
1,6,6.0,NaN,8.0,6.0
2,8,6.0,NaN,6.0,2.0
3,6,8.0,6.0,6.0,6.0
4,6,6.0,6.0,2.0,6.0
5,2,6.0,8.0,6.0,6.0
6,6,2.0,6.0,6.0,32.0
7,6,6.0,6.0,6.0,6.0
8,6,6.0,2.0,32.0,6.0
9,32,6.0,6.0,6.0,8.0
